# Tokyo demographics

In [1]:
import os

APP_ID = os.environ['APP_ID']
LANG = 'J'
BASE_URL = 'https://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?appId={id}&lang={lang}'.format(id=APP_ID,
                                                                                                    lang=LANG)

In [6]:
import requests
import pandas
from datetime import datetime

json = requests.get('{url}&statsDataId=0003411563'.format(url=BASE_URL)).json()
stats = json['GET_STATS_DATA']['STATISTICAL_DATA']
labels = stats['CLASS_INF']['CLASS_OBJ']
labelMap = {label['@id']: {l['@code']: l['@name'] for l in label['CLASS']} for label in labels}
data = stats['DATA_INF']['VALUE']
for d in data:
    d['@cat01'] = labelMap['cat01'][d['@cat01']]
    d['@area'] = labelMap['area'][d['@area']]
    d['@time'] = datetime.strptime(labelMap['time'][d['@time']], '%Y年')
df = pandas.DataFrame(data)
df = df.rename(columns={'@cat01': 'Type', '@area': 'Area', '@time': 'Year', '@unit': 'Unit', '$': 'Value'})
df['Value'] = df['Value'].apply(pandas.to_numeric, errors='coerce')
df

,Type,Area,Year,Unit,Value
0,出生数,全国,2021-01-01,人,811622.00
1,出生数,全国,2020-01-01,人,840835.00
2,出生数,全国,2019-01-01,人,865239.00
3,出生数,全国,2018-01-01,人,918400.00
4,出生数,全国,2017-01-01,人,946146.00
...,...,...,...,...,...
29815,離婚率,熊本市,2019-01-01,人口千対,1.79
29816,離婚率,熊本市,2018-01-01,人口千対,1.76
29817,離婚率,熊本市,2017-01-01,人口千対,1.62
29818,離婚率,熊本市,2016-01-01,人口千対,1.70
